# Lesson 4: Structured Outputs

**🟡 Intermediate · ⏱ 25 min**

---

By default agents return free-form text. Structured outputs let you define exactly what shape the response should take using Pydantic models — giving you type safety, IDE autocomplete, and easy downstream processing.

## What you will learn

- Define Pydantic response models with field descriptions
- Get typed agent responses instead of raw strings
- Work with nested models for hierarchical data
- Handle optional fields gracefully

## Prerequisites

Before starting this lesson, make sure you have completed:

- **Lesson 1** — Getting Started: `Studio`, `create_agent`, `agent.run`
- **Lesson 2** — Providers and Models
- **Lesson 3** — Agent Lifecycle

`pydantic` is included with `lyzr-adk`, so no separate install is needed. If you are running in a fresh environment, the next cell handles everything.

In [ ]:
!pip install lyzr-adk pydantic -q

In [ ]:
import os
from lyzr import Studio
from pydantic import BaseModel, Field
from typing import List, Optional

API_KEY = os.getenv("LYZR_API_KEY", "YOUR_LYZR_API_KEY")
studio = Studio(api_key=API_KEY)
print("Ready!")

## Why Structured Outputs?

When an agent returns free-form text, consuming that output in code requires fragile string parsing:

```python
# Without structured outputs — brittle
response = agent.run("What is the rating of Inception?")
text = response.response  # "I'd rate Inception an 8.7 out of 10..."
# Now you need regex or LLM parsing to extract 8.7
```

Structured outputs solve this by having the agent return a Pydantic model instance directly:

| Feature | Free-form text | Structured output |
|---------|---------------|-------------------|
| Type safety | No | Yes |
| IDE autocomplete | No | Yes |
| Validation | No | Yes (Pydantic) |
| Downstream processing | Fragile parsing | Direct field access |
| Agent guidance | Prompt only | Field names + descriptions |

The field names and `Field(description=...)` values are passed to the agent as guidance, so descriptive field names and clear descriptions lead to more accurate extraction.

## Defining Your First Response Model

Create a Pydantic `BaseModel` subclass. Use `Field(description=...)` to tell the agent exactly what each field should contain — the richer the description, the more accurate the result.

In [ ]:
class MovieReview(BaseModel):
    title: str = Field(description="The exact title of the movie")
    year: int = Field(description="Release year")
    rating: float = Field(description="Rating out of 10.0")
    summary: str = Field(description="One sentence plot summary")
    pros: List[str] = Field(description="List of 3 things the movie does well")
    cons: List[str] = Field(description="List of 2 weaknesses")
    recommended: bool = Field(description="Whether you recommend watching it")

print("Model defined:", list(MovieReview.model_fields.keys()))

## Creating a Specialized Agent

The agent's `role`, `goal`, and `instructions` work together with the response model's field descriptions. A focused agent with clear instructions produces more consistent structured responses.

In [ ]:
reviewer = studio.create_agent(
    name="Movie Critic",
    provider="openai/gpt-4o",
    role="Professional film critic",
    goal="Provide structured movie reviews",
    instructions="Always fill in all fields completely and accurately."
)
print(f"Reviewer agent created: {reviewer.id}")

## Getting a Structured Response

Pass your Pydantic model class as `response_format` to `agent.run`. The key difference from previous lessons:

- **Without** `response_format`: `response.response` is a `str`
- **With** `response_format=MovieReview`: `response.response` is a `MovieReview` instance

You get full IDE autocomplete and type checking on the returned object.

In [ ]:
response = reviewer.run(
    "Review the movie Inception (2010)",
    response_format=MovieReview
)

# response.response is now a MovieReview object, not a string
review: MovieReview = response.response

print(f"Title:       {review.title} ({review.year})")
print(f"Rating:      {review.rating}/10")
print(f"Summary:     {review.summary}")
print(f"Pros:        {', '.join(review.pros)}")
print(f"Cons:        {', '.join(review.cons)}")
print(f"Recommended: {'Yes ✅' if review.recommended else 'No ❌'}")

## Nested Models

Pydantic models can contain other models, enabling hierarchical data structures. This is particularly useful for extraction tasks where you need to pull multiple entities from a single piece of text.

```
ExtractedPeople
├── total_count: int
└── people: List[Person]
    ├── name: str
    ├── age: Optional[int]
    └── occupation: str
```

The agent understands the nesting and populates each level correctly.

In [ ]:
class Person(BaseModel):
    name: str = Field(description="Full name of the person")
    age: Optional[int] = Field(None, description="Age if mentioned, otherwise None")
    occupation: str = Field(description="Their job or role")

class ExtractedPeople(BaseModel):
    people: List[Person] = Field(description="All people mentioned in the text")
    total_count: int = Field(description="Total number of people found")

extractor = studio.create_agent(
    name="Entity Extractor",
    provider="openai/gpt-4o",
    role="Information extraction specialist",
    goal="Extract structured people data from text",
    instructions="Extract every person mentioned. If age is not stated, leave it as null."
)

text = "Alice Chen (35, software engineer) and Bob Martinez (designer, mid-30s) co-founded the startup with Dr. Sarah Kim."
result = extractor.run(text, response_format=ExtractedPeople)
data: ExtractedPeople = result.response

print(f"Found {data.total_count} people:")
for person in data.people:
    age_str = str(person.age) if person.age else "unknown"
    print(f"  • {person.name}, age {age_str}, {person.occupation}")

## Common Mistake: Treating a Structured Response as a String

When you pass `response_format`, `response.response` is **no longer a string**. A common error is calling string methods on the typed object.

The cell below demonstrates the error and the correct fix.

In [ ]:
# ❌ Common mistake: treating structured response as a string
response3 = reviewer.run("Review The Matrix (1999)", response_format=MovieReview)

try:
    # This fails — it's a MovieReview object, not a string
    print(response3.response.upper())
except AttributeError as e:
    print(f"Error (expected): {e}")

# ✅ Correct: access fields directly
matrix: MovieReview = response3.response
print(f"✅ Rating: {matrix.rating}/10")

## Exercise

Define a `ProductSummary` Pydantic model with the following fields:

| Field | Type | Description |
|-------|------|-------------|
| `name` | `str` | Product name |
| `price_usd` | `float` | Price in US dollars |
| `category` | `str` | Product category |
| `features` | `List[str]` | Key product features |
| `in_stock` | `bool` | Whether the product is currently in stock |

Then create an extraction agent and extract structured data from the product description below.

> **Hint:** Write a descriptive `Field(description=...)` for each field — this directly guides what the agent extracts.

In [ ]:
# TODO: Define a ProductSummary Pydantic model
class ProductSummary(BaseModel):
    name: str = Field(description=...)
    price_usd: float = Field(description=...)
    category: str = Field(description=...)
    features: List[str] = Field(description=...)
    in_stock: bool = Field(description=...)

# TODO: Create an agent to extract product info
product_agent = studio.create_agent(
    name=...,
    provider="openai/gpt-4o",
    role=...,
    goal=...,
    instructions=...
)

# TODO: Extract structured data from this description
description = """
The TurboBlend Pro 3000 is a high-performance blender priced at $149.99.
It features a 1200W motor, 6 stainless steel blades, 64oz BPA-free container,
and 5 speed settings. Currently in stock in the Kitchen Appliances category.
"""

result = product_agent.run(description, response_format=ProductSummary)
product: ProductSummary = result.response
print(f"Product: {product.name}")
print(f"Price: ${product.price_usd}")
# ... print other fields

## Summary

### Structured vs Unstructured Output

| | Unstructured | Structured |
|---|---|---|
| `response_format` | Not passed | `response_format=MyModel` |
| `response.response` type | `str` | `MyModel` instance |
| Field access | String parsing | `obj.field_name` |
| Type safety | None | Full Pydantic validation |
| Optional fields | Manual handling | `Optional[T] = None` |
| Nested data | Complex parsing | Nested `BaseModel` |

### Key Takeaways

1. **Define models with `Field(description=...)`** — descriptions guide the agent, not just document your code.
2. **`response.response` changes type** — with `response_format` it is a typed object, not a string.
3. **Nest models freely** — `List[MyModel]` and nested `BaseModel` fields work as expected.
4. **Use `Optional[T] = None`** for fields that may not always be present in the source text.
5. **Agent instructions still matter** — tell the agent explicitly what to do when fields are absent or ambiguous.

## Next Steps

**Lesson 5: Memory and Sessions** — Learn how to give agents persistent memory so they can maintain context across multiple turns of a conversation.

---

*Lesson 4 of 10 · lyzr-adk fundamentals series*